# sudo no password

In [ ]:
# sudo no password
sudo grep -q "^$USER ALL=(ALL) NOPASSWD: ALL" /etc/sudoers || echo "$USER ALL=(ALL) NOPASSWD: ALL" | sudo tee -a /etc/sudoers
# restart vscode

# Miniconda3

In [ ]:
%%bash
# wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh      
# need to input enter 
rm -rf $(pwd)/bin/miniconda3
sh ./Miniconda3-latest-Linux-x86_64.sh -b -p $(pwd)/bin/miniconda3
# Initialize conda for zsh
$(pwd)/bin/miniconda3/bin/conda init zsh
rm ./Miniconda3-latest-Linux-x86_64.sh
source ~/.zshrc
# Add conda-forge channel for more packages
conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main
conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r
conda install jupyter ipykernel -y

# restart vscode

# VSCode

In [ ]:
%%bash
# 下载 VSCode
wget -O vscode.tar.gz "https://code.visualstudio.com/sha/download?build=stable&os=linux-x64"

# 解压并移动到 ./bin/
mkdir -p ./bin
rm -rf ./bin/vscode
tar -xzf vscode.tar.gz 
mv $(tar -tzf vscode.tar.gz | head -1 | cut -f1 -d"/") ./bin/vscode


In [ ]:

# 添加到 zsh PATH
import os

def add_path_to_zsh(path, zsh_config_path="~/.zshrc", position="append", check_exists=True):
    """
    Add a path to zsh configuration file with options
    
    Args:
        path (str): The path to add to PATH
        zsh_config_path (str): Path to zsh config file
        position (str): 'append' (path:$PATH) or 'prepend' ($PATH:path)
        check_exists (bool): Check if path already exists in file
    """
    # Expand the tilde to absolute path
    zsh_config_path = os.path.expanduser(zsh_config_path)
    path = os.path.expanduser(path)
    
    # Check if config file exists
    if not os.path.exists(zsh_config_path):
        print(f"Error: Config file {zsh_config_path} does not exist")
        return False
    
    # Check if the path exists
    if not os.path.exists(path):
        print(f"Warning: Path {path} does not exist on filesystem")
    
    # Read current content
    with open(zsh_config_path, 'r') as file:
        content = file.read()
    
    # Create the export line based on position
    if position == "prepend":
        export_line = f'export PATH="{path}:$PATH"'
    else:  # append
        export_line = f'export PATH="$PATH:{path}"'
    
    # Check if path already exists
    if check_exists:
        # Check for various formats
        existing_patterns = [
            f'export PATH="{path}:',
            f'export PATH="$PATH:{path}"',
            f'export PATH="{path}:$PATH"',
            f'PATH="{path}:',
            f'PATH="$PATH:{path}"'
        ]
        
        if any(pattern in content for pattern in existing_patterns):
            print(f"Path {path} already exists in {zsh_config_path}")
            return True
    
    # Add the path to the file
    with open(zsh_config_path, 'a') as file:
        file.write(f'\n# Added by script\n')
        file.write(f'{export_line}\n')
    
    print(f"Successfully added {path} to {zsh_config_path}")
    print("Please run 'source {}' to apply changes".format(zsh_config_path))
    return True

if __name__ == "__main__":
    # Basic usage
    code_path=os.path.join(os.getcwd(), 'bin', 'vscode')
    add_path_to_zsh(code_path)

    # Custom zsh config file
    # add_path_to_zsh("/opt/myapp/bin", "~/.config/zsh/custom_paths.zsh")

!source ~/.zshrc
!cat ~/.zshrc

## pip

In [ ]:
import os
from pathlib import Path
def create_pip_conf(pip_dir=None):
    """
    Create pip configuration file with USTC mirror
    
    Args:
        pip_dir (str): Directory where pip.conf will be created
                      (default: ~/.config/pip)
    """

    
    if pip_dir is None:
        pip_dir = os.path.join(os.path.expanduser("~"), ".config", "pip")
    
    # Create directory if it doesn't exist
    pip_path = Path(pip_dir)
    pip_path.mkdir(parents=True, exist_ok=True)
    
    # Create pip.conf content
    pip_conf_content = """[global]
index-url = https://mirrors.ustc.edu.cn/pypi/web/simple
timeout = 120
trusted-host = mirrors.ustc.edu.cn

[install]
trusted-host = mirrors.ustc.edu.cn
"""
    
    # Write pip.conf file
    pip_conf_file = pip_path / "pip.conf"
    with open(pip_conf_file, 'w') as f:
        f.write(pip_conf_content)
    
    print(f"✅ Created pip config: {pip_conf_file}")
    return str(pip_conf_file)



if __name__ == "__main__" :
    pip_dir=os.path.join(os.getcwd(),"user",".pip")
    create_pip_conf(pip_dir)

In [ ]:
import os
import shutil
from pathlib import Path

class LinkManager:
    """Manage symbolic links with additional utilities"""
    
    @staticmethod
    def create_link(source_dir, target_dir, create_source=True, overwrite=True):
        """
        Create symbolic link with comprehensive options
        """
        try:
            source = Path(source_dir)
            target = Path(target_dir)
            
            # Check if target exists and handle accordingly
            if target.exists():
                if not overwrite:
                    print(f"⚠️  Target exists and overwrite=False: {target_dir}")
                    return False
                
                if target.is_symlink():
                    target.unlink()
                elif target.is_dir():
                    shutil.rmtree(target)
                else:
                    target.unlink()
                print(f"🗑️  Removed existing target: {target_dir}")
            
            # Create source directory if needed
            if create_source and not source.exists():
                source.mkdir(parents=True, exist_ok=True)
                print(f"📁 Created source directory: {source_dir}")
            
            # Create the symbolic link
            target.symlink_to(source.resolve())
            print(f"✅ Created symbolic link: {target_dir} -> {source_dir}")
            return True
            
        except Exception as e:
            print(f"❌ Failed to create symbolic link: {e}")
            return False
    
    @staticmethod
    def check_link(target_dir):
        """
        Comprehensive link checking
        """
        try:
            target = Path(target_dir)
            
            if not target.exists():
                return {
                    'status': 'missing',
                    'message': f"❌ Target does not exist: {target_dir}",
                    'valid': False
                }
            
            if target.is_symlink():
                link_target = target.readlink()
                target_exists = link_target.exists()
                
                if target_exists:
                    return {
                        'status': 'valid',
                        'message': f"✅ Valid symbolic link: {target_dir} -> {link_target}",
                        'target': str(link_target),
                        'target_exists': True,
                        'valid': True
                    }
                else:
                    return {
                        'status': 'broken',
                        'message': f"⚠️  Broken symbolic link: {target_dir} -> {link_target} (target missing)",
                        'target': str(link_target),
                        'target_exists': False,
                        'valid': False
                    }
            else:
                item_type = 'directory' if target.is_dir() else 'file'
                return {
                    'status': 'not_link',
                    'message': f"❌ Not a symbolic link: {target_dir} (it's a {item_type})",
                    'type': item_type,
                    'valid': False
                }
                
        except Exception as e:
            return {
                'status': 'error',
                'message': f"❌ Error checking link: {e}",
                'valid': False,
                'error': str(e)
            }
    
    @staticmethod
    def bulk_create_links(links, create_sources=True):
        """
        Create multiple symbolic links at once
        
        Args:
            links (list): List of (source, target) tuples
            create_sources (bool): Whether to create source directories
        """
        results = []
        for source, target in links:
            success = LinkManager.create_link(source, target, create_sources)
            results.append({
                'source': source,
                'target': target,
                'success': success
            })
        return results

# Usage examples
if __name__ == "__main__":
    source_dir=os.path.join(os.getcwd(), 'user', '.pip')
    target_dir="~/.pip"
    # Using the class
    LinkManager.create_link(source_dir, os.path.expanduser(target_dir))
    
    # # Bulk creation
    # links = [
    #     (pip_dir, os.path.expanduser("~/.pip"))
    # ]
    # LinkManager.bulk_create_links(links)

# ssh

In [ ]:
import os
import subprocess
from pathlib import Path

def setup_ssh_environment(email="19157521820@163.com", ssh_dir=None):

    """
    Setup SSH environment similar to the shell script
    
    Args:
        email (str): Email for SSH key generation
        ssh_dir (str): SSH directory path (default: current_dir/user/.ssh)
    """
    # Get current directory absolute path
    current_dir = os.getcwd()
    
    # Construct SSH directory path
    if ssh_dir is None:
        ssh_dir = os.path.join(current_dir, "user", ".ssh")
    
    # Create SSH directory if it doesn't exist
    ssh_path = Path(ssh_dir)
    ssh_path.mkdir(parents=True, exist_ok=True)
    print(f"📁 Created SSH directory: {ssh_dir}")
    
    # Check if SSH key exists, generate if not
    private_key_path = ssh_path / "id_ed25519"
    if not private_key_path.exists():
        try:
            subprocess.run([
                "ssh-keygen", "-t", "ed25519", "-C", email,
                "-f", str(private_key_path), "-N", "", "-q"
            ], check=True)
            print("✅ Generated new SSH key pair")
            
            # Display public key
            public_key_path = ssh_path / "id_ed25519.pub"
            if public_key_path.exists():
                with open(public_key_path, 'r') as f:
                    print(f"🔑 Public key: {f.read().strip()}")
                    
        except subprocess.CalledProcessError as e:
            print(f"❌ Failed to generate SSH key: {e}")
            return False
    else:
        print("✅ Using existing SSH key pair")
    
    # Create config file if it doesn't exist
    config_path = ssh_path / "config"
    if not config_path.exists():
        config_content = """# Global SSH configuration
Host *
    ServerAliveInterval 60
    ServerAliveCountMax 3
    TCPKeepAlive yes
    Compression yes
    ForwardAgent yes
    AddKeysToAgent yes
"""
        with open(config_path, 'w') as f:
            f.write(config_content)
        print("✅ Created SSH config file")
    
    return True


if __name__ =="__main__":
    setup_ssh_environment()
    source_dir=os.path.join(os.getcwd(), 'user', '.ssh')
    target_dir="~/.ssh"
    # Using the class
    LinkManager.create_link(source_dir, os.path.expanduser(target_dir))

# fonts

In [ ]:
if __name__ =="__main__":
    source_dir=os.path.join(os.getcwd(), 'user', '.fonts')
    target_dir="~/.fonts"
    # Using the class
    LinkManager.create_link(source_dir, os.path.expanduser(target_dir))

# other link

In [ ]:
import os
from pathlib import Path

# Chinese to English directory name mapping
CHINESE_TO_ENGLISH = {
    "视频": "Videos",
    "图片": "Pictures", 
    "文档": "Documents",
    "下载": "Downloads",
    "音乐": "Music"
}

def convert_directory_names(base_path):
    """
    Convert Chinese directory names to English standard names
    
    Args:
        base_path (str): Base path containing Chinese directories
    """
    base_path = Path(base_path)
    
    for chinese_name, english_name in CHINESE_TO_ENGLISH.items():
        chinese_dir = base_path / chinese_name
        english_dir = base_path / english_name
        
        # If Chinese directory exists, rename to English
        if chinese_dir.exists() and not english_dir.exists():
            chinese_dir.rename(english_dir)
            print(f"✅ Renamed: {chinese_name} -> {english_name}")
        elif chinese_dir.exists():
            print(f"⚠️  Both {chinese_name} and {english_name} exist, skipping")
        elif english_dir.exists():
            print(f"✅ English directory already exists: {english_name}")
        else:
            # Create English directory if neither exists
            english_dir.mkdir(parents=True, exist_ok=True)
            print(f"📁 Created: {english_name}")

# Usage examples
if __name__ == "__main__":
    # Convert existing Chinese directories to English
    dir=os.path.join(os.getcwd(),"home","user")
    convert_directory_names(dir)
    # Get current directory
    current_dir = Path.cwd()
    base_path = current_dir / "user"

    # Define all user directory paths
    Documents_path = base_path / "Documents"
    Pictures_path = base_path / "Pictures" 
    Musics_path = base_path / "Music"  # Note: Usually "Music" not "Musics"
    Downloads_path = base_path / "Downloads"
    Videos_path = base_path / "Videos"

    # Create links to home directory
    links = [
        (Documents_path, os.path.expanduser("~/Documents")),
        (Pictures_path, os.path.expanduser("~/Pictures")),
        (Musics_path, os.path.expanduser("~/Music")),
        (Downloads_path, os.path.expanduser("~/Downloads")),
        (Videos_path, os.path.expanduser("~/Videos"))
    ]

    # Create the links
    LinkManager.bulk_create_links(links)


# pacman-mirrors

In [ ]:
%%bash
#!/bin/bash
# 开始配置china镜像源
sudo pacman-mirrors -c China
sudo pacman-mirrors --fasttrack 5
# 刷新包数据库
sudo pacman -Syy
# 检查并添加 archlinuxcn 源
sudo sh -c 'grep -q "archlinuxcn" /etc/pacman.conf || (echo -e "\n[archlinuxcn]\nSigLevel = Optional TrustAll\nServer = https://mirrors.ustc.edu.cn/archlinuxcn/\$arch" >> /etc/pacman.conf && echo "archlinuxcn 镜像源添加成功" || echo "archlinuxcn 镜像源添加失败")'

# archlinuxcn-keyring

In [ ]:
# 6. Pacman密钥
!sudo pacman -Sy --needed --noconfirm archlinuxcn-keyring && sudo pacman-key --init && sudo pacman-key --populate manjaro archlinux archlinuxcn && sudo pacman -Syyu --noconfirm

# git

In [ ]:
# git ssh pip
!git config --global user.email "19157521820@163.com" && git config --global user.name "kefu"


# fcitx5

In [ ]:
# 7. 输入法
!sudo pacman -Sy --needed --noconfirm manjaro-asian-input-support-fcitx5 fcitx5 fcitx5-configtool fcitx5-chinese-addons && echo -e "export GTK_IM_MODULE=fcitx\nexport QT_IM_MODULE=fcitx\nexport XMODIFIERS=@im=fcitx" > ~/.xprofile

# 软件包安装

In [ ]:
%%bash
# 定义软件包列表变量
BASE_PACKAGES="base-devel yay git-lfs haveged"
MEDIA_PACKAGES="vlc ffmpeg qbittorrent"
FONT_PACKAGES="adobe-source-han-sans-otc-fonts adobe-source-han-serif-otc-fonts wqy-zenhei noto-fonts-cjk unzip-natspec"
APP_PACKAGES="chromium telegram-desktop  partitionmanager clash-verge-rev discord  aria2  docker docker-compose obs-studio onlyoffice-desktopeditors"

# 安装函数
install_packages() {
    local category=$1
    local packages=$2
    
    echo "安装 $category: $packages"
    sudo pacman -Sy --needed --noconfirm $packages
}

# 安装所有软件包
install_packages "基础工具" "$BASE_PACKAGES"
install_packages "多媒体工具" "$MEDIA_PACKAGES" 
install_packages "字体包" "$FONT_PACKAGES"
install_packages "应用程序" "$APP_PACKAGES"

sudo systemctl enable haveged


In [ ]:
%%bash
AUR_PACKAGES="google-chrom"
install_yay_packages() {
    local category=$1
    local packages=$2
    
    echo "安装 $category: $packages"
    yay -Sy --needed --noconfirm $packages
}
install_yay_packages "AUR软件包" "$AUR_PACKAGES"

# add path to zsh

In [ ]:
bin_dir=os.path.join(os.getcwd(),"bin")
add_path_to_zsh(bin_dir)


# add brtfs part to fstab

In [ ]:
%%bash
lsblk -f
sudo mkdir -p /data && echo "UUID=180161ab-546d-4dfe-969e-e1cf9938195e /data btrfs defaults,compress=zstd,noatime 0 0" | sudo tee -a /etc/fstab

systemctl daemon-reload
sudo mount -a
df -h | grep /data
sudo chown $USER:$USER /data
sudo chmod 755 /data


# system config
## dolphin
- 详情
- 隐藏文件
- 终端
- 右键vscode打开
## yakuake


In [ ]:
#!/usr/bin/env python3
"""
检测 XDG 目录和 Dolphin 常用位置是否为中文，转换为英文
"""

import os
import re
import subprocess
import shutil
from pathlib import Path

def get_xdg_config():
    """读取 XDG 用户目录配置"""
    xdg_config_file = Path.home() / '.config' / 'user-dirs.dirs'
    if not xdg_config_file.exists():
        return None
    
    config = {}
    with open(xdg_config_file, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '=' in line:
                key, value = line.split('=', 1)
                key = key.strip()
                value = value.strip().strip('"')
                # 处理 $HOME 变量
                if value.startswith('$HOME/'):
                    value = str(Path.home() / value[6:])
                config[key] = value
    return config

def get_dolphin_places():
    """读取 Dolphin 常用位置配置"""
    kdeglobals_file = Path.home() / '.config' / 'kdeglobals'
    if not kdeglobals_file.exists():
        return None
    
    places = {}
    in_user_places = False
    count = 0
    
    with open(kdeglobals_file, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line == '[UserPlaces]':
                in_user_places = True
                continue
            elif in_user_places and line.startswith('['):
                break
                
            if in_user_places and '=' in line:
                key, value = line.split('=', 1)
                key = key.strip()
                value = value.strip()
                
                if key == 'Count':
                    count = int(value)
                elif '[path]' in key:
                    place_num = key.split('[')[0].replace('place', '')
                    if place_num not in places:
                        places[place_num] = {}
                    places[place_num]['path'] = value
                elif '[title]' in key:
                    place_num = key.split('[')[0].replace('place', '')
                    if place_num not in places:
                        places[place_num] = {}
                    places[place_num]['title'] = value
                elif '[icon]' in key:
                    place_num = key.split('[')[0].replace('place', '')
                    if place_num not in places:
                        places[place_num] = {}
                    places[place_num]['icon'] = value
    
    return places

def is_chinese_text(text):
    """检测文本是否包含中文字符"""
    if not text:
        return False
    chinese_pattern = re.compile(r'[\u4e00-\u9fff]')
    return bool(chinese_pattern.search(str(text)))

def get_english_mapping():
    """返回中文到英文的目录名映射"""
    return {
        '桌面': 'Desktop',
        '下载': 'Downloads', 
        '文档': 'Documents',
        '音乐': 'Music',
        '图片': 'Pictures',
        '视频': 'Videos',
        '模板': 'Templates',
        '公共': 'Public'
    }

def convert_xdg_to_english(config):
    """将中文 XDG 目录转换为英文"""
    home = Path.home()
    mapping = get_english_mapping()
    converted = False
    new_config = {}
    
    for key, path in config.items():
        if not path.startswith(str(home)):
            new_config[key] = path
            continue
            
        dir_name = Path(path).name
        if dir_name in mapping:
            english_name = mapping[dir_name]
            new_path = home / english_name
            
            # 如果原目录存在且是中文，重命名它
            old_path = Path(path)
            if old_path.exists() and is_chinese_text(dir_name):
                print(f"重命名 XDG 目录: {old_path} -> {new_path}")
                try:
                    shutil.move(str(old_path), str(new_path))
                    converted = True
                except Exception as e:
                    print(f"重命名失败: {e}")
                    new_config[key] = f"$HOME/{english_name}"
                else:
                    new_config[key] = f"$HOME/{english_name}"
            else:
                new_config[key] = f"$HOME/{english_name}"
        else:
            new_config[key] = path
    
    return new_config, converted

def convert_dolphin_places_to_english(places):
    """将 Dolphin 常用位置转换为英文"""
    mapping = get_english_mapping()
    converted = False
    new_places = {}
    
    for place_num, place_info in places.items():
        path = place_info.get('path', '')
        title = place_info.get('title', '')
        icon = place_info.get('icon', 'folder')
        
        new_path = path
        new_title = title
        
        # 检查路径是否包含中文
        if is_chinese_text(path):
            for chinese, english in mapping.items():
                if chinese in path:
                    new_path = path.replace(chinese, english)
                    print(f"转换 Dolphin 路径: {path} -> {new_path}")
                    converted = True
                    break
        
        # 检查标题是否包含中文
        if is_chinese_text(title):
            for chinese, english in mapping.items():
                if chinese == title:
                    new_title = english
                    print(f"转换 Dolphin 标题: {title} -> {new_title}")
                    converted = True
                    break
        
        new_places[place_num] = {
            'path': new_path,
            'title': new_title,
            'icon': icon
        }
    
    return new_places, converted

def update_xdg_config(config):
    """更新 XDG 配置文件"""
    xdg_config_file = Path.home() / '.config' / 'user-dirs.dirs'
    
    content = [
        '# This file is written by xdg-user-dirs-update',
        '# If you want to change or add directories, just edit the line you\'re',
        '# interested in. All local changes will be retained on the next run.',
        '# Format is XDG_xxx_DIR="$HOME/yyy", where yyy is a shell-escaped',
        '# homedir-relative path, or XDG_xxx_DIR="/yyy", where /yyy is an',
        '# absolute path. No other format is supported.',
        '#'
    ]
    
    # 按特定顺序写入配置
    keys_order = [
        'XDG_DESKTOP_DIR', 'XDG_DOWNLOAD_DIR', 'XDG_TEMPLATES_DIR',
        'XDG_PUBLICSHARE_DIR', 'XDG_DOCUMENTS_DIR', 'XDG_MUSIC_DIR',
        'XDG_PICTURES_DIR', 'XDG_VIDEOS_DIR'
    ]
    
    for key in keys_order:
        if key in config:
            content.append(f'{key}="{config[key]}"')
    
    with open(xdg_config_file, 'w', encoding='utf-8') as f:
        f.write('\n'.join(content) + '\n')
    
    print("XDG 配置文件已更新")

def update_dolphin_places(places):
    """更新 Dolphin 常用位置"""
    # 设置常用位置总数
    subprocess.run([
        'kwriteconfig5', '--file', 'kdeglobals', 
        '--group', 'UserPlaces', '--key', 'Count', str(len(places))
    ], check=True)
    
    # 添加每个位置
    for place_num, place_info in places.items():
        path = place_info.get('path', '')
        title = place_info.get('title', '')
        icon = place_info.get('icon', 'folder')
        
        # 确保目录存在
        if path.startswith(str(Path.home())):
            os.makedirs(path, exist_ok=True)
        
        subprocess.run([
            'kwriteconfig5', '--file', 'kdeglobals',
            '--group', 'UserPlaces', '--key', f'place{place_num}[enabled]', 'true'
        ], check=True)
        
        subprocess.run([
            'kwriteconfig5', '--file', 'kdeglobals',
            '--group', 'UserPlaces', '--key', f'place{place_num}[icon]', icon
        ], check=True)
        
        subprocess.run([
            'kwriteconfig5', '--file', 'kdeglobals',
            '--group', 'UserPlaces', '--key', f'place{place_num}[path]', path
        ], check=True)
        
        subprocess.run([
            'kwriteconfig5', '--file', 'kdeglobals',
            '--group', 'UserPlaces', '--key', f'place{place_num}[title]', title
        ], check=True)
    
    print("Dolphin 常用位置已更新")

def create_default_dolphin_places():
    """创建默认的 Dolphin 常用位置"""
    home = str(Path.home())
    
    places = {
        '1': {'path': f"{home}/Desktop", 'icon': 'user-desktop', 'title': 'Desktop'},
        '2': {'path': f"{home}/Downloads", 'icon': 'folder-download', 'title': 'Downloads'},
        '3': {'path': f"{home}/Documents", 'icon': 'folder-documents', 'title': 'Documents'},
        '4': {'path': f"{home}/Pictures", 'icon': 'folder-image', 'title': 'Pictures'},
        '5': {'path': f"{home}/Music", 'icon': 'folder-music', 'title': 'Music'},
        '6': {'path': f"{home}/Videos", 'icon': 'folder-video', 'title': 'Videos'},
        '7': {'path': f"{home}/Templates", 'icon': 'folder-text', 'title': 'Templates'},
        '8': {'path': f"{home}/Public", 'icon': 'folder-publicshare', 'title': 'Public'}
    }
    
    return places

def run_xdg_update():
    """运行 xdg-user-dirs-update"""
    try:
        subprocess.run(['xdg-user-dirs-update'], check=True)
        print("XDG 目录更新完成")
    except subprocess.CalledProcessError as e:
        print(f"XDG 更新失败: {e}")

def restart_dolphin():
    """重启 Dolphin"""
    try:
        subprocess.run(['killall', 'dolphin'], capture_output=True)
        subprocess.Popen(['dolphin'])
        print("Dolphin 已重启")
    except Exception as e:
        print(f"重启 Dolphin 失败: {e}")

def main():
    """主函数"""
    print("=== 检测和转换 XDG 目录与 Dolphin 常用位置 ===\n")
    
    xdg_converted = False
    dolphin_converted = False
    
    # 1. 检查和处理 XDG 目录
    print("1. 检查 XDG 目录...")
    xdg_config = get_xdg_config()
    if xdg_config:
        print("当前 XDG 配置:")
        for key, value in xdg_config.items():
            chinese_flag = " (中文)" if is_chinese_text(value) else ""
            print(f"  {key}: {value}{chinese_flag}")
        
        new_xdg_config, xdg_converted = convert_xdg_to_english(xdg_config)
        if xdg_converted:
            update_xdg_config(new_xdg_config)
            run_xdg_update()
    else:
        print("未找到 XDG 配置文件")
    
    # 2. 检查和处理 Dolphin 常用位置
    print("\n2. 检查 Dolphin 常用位置...")
    dolphin_places = get_dolphin_places()
    
    if dolphin_places:
        print("当前 Dolphin 常用位置:")
        for place_num, place_info in dolphin_places.items():
            path = place_info.get('path', '')
            title = place_info.get('title', '')
            path_chinese = " (路径中文)" if is_chinese_text(path) else ""
            title_chinese = " (标题中文)" if is_chinese_text(title) else ""
            print(f"  位置 {place_num}: 路径={path}{path_chinese}, 标题={title}{title_chinese}")
        
        new_dolphin_places, dolphin_converted = convert_dolphin_places_to_english(dolphin_places)
    else:
        print("未找到 Dolphin 常用位置配置，将创建默认配置")
        new_dolphin_places = create_default_dolphin_places()
        dolphin_converted = True
    
    # 3. 如果没有转换但需要更新，询问用户
    if not xdg_converted and not dolphin_converted:
        print("\n没有发现需要转换的中文目录")
        
        update_choice = input("是否更新 Dolphin 常用位置为默认英文配置? (y/N): ")
        if update_choice.lower() in ['y', 'yes']:
            new_dolphin_places = create_default_dolphin_places()
            dolphin_converted = True
    
    # 4. 更新 Dolphin 常用位置（如果需要）
    if dolphin_converted:
        print("\n=== 更新 Dolphin 常用位置 ===")
        update_dolphin_places(new_dolphin_places)
        
        # 重启 Dolphin
        restart_dolphin()
        
        print("\n=== 完成 ===")
        if xdg_converted:
            print("✓ XDG 中文目录已转换为英文")
        if dolphin_converted:
            print("✓ Dolphin 常用位置已更新为英文")
    else:
        print("\n无需更新，所有配置已经是英文")

if __name__ == "__main__":
    main()